# Video processing with OpenCV

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import time
import serial

# %matplotlib inline

In [2]:
# Specify camera/device number (usually 0)
camera = 0

# Specifiy number of frames to acquire
num_frames = 1000

# Open camera stream
cam = cv2.VideoCapture(camera)

In [3]:
cam.isOpened()

True

In [4]:
kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))

min_perimeter = 15

# Make space for intermediate images (of correct data type)
# gray = np.zeros((height, width), dtype = np.uint8)
# gray_float = np.zeros((height, width), dtype = np.float32)
# previous = np.zeros((height, width), dtype = np.float32)
# difference = np.zeros((height, width), dtype = np.float32)

# cv2.namedWindow('image', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('image', 600, 600)

In [5]:
# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

# # Change thresholds
# params.minThreshold = 10
# params.maxThreshold = 200
 
# Filter by Area.
params.filterByArea = False # coulde be faster without it
params.minArea = 1500
 
# # Filter by Circularity
params.filterByCircularity = False
# params.minCircularity = 0.1
 
# # Filter by Convexity
params.filterByConvexity = False
# params.minConvexity = 0.87
 
# # Filter by Inertia
params.filterByInertia = False
# params.minInertiaRatio = 0.01
 
# Create a detector with the parameters
detector = cv2.SimpleBlobDetector_create(params)

https://www.learnopencv.com/read-write-and-display-a-video-using-opencv-cpp-python/

In [6]:
# def get_transformed_image(image, anchor_points, new_anchor_points):
#     rows, cols, _ = image.shape

#     new_points = new_anchor_points
    
#     matrix = cv2.getAffineTransform(new_anchor_points, anchor_points)
    
#     return cv2.warpAffine(image, matrix, (cols, rows))

In [43]:
# def get_anchor_points(init_points):
#     distances = np.sum(init_points ** 2, axis=0)
#     first_point_ind = np.argmin(distances)
    
#     distanes = np.sum((init_points - 
#                        np.repeat(init_points[:, first_point_ind].reshape((2, 1)), 
#                                  4, axis=1)) ** 2, axis=0)
#     indices = np.argsort(distances)
#     np.sort(np.sqrt(distances))
    
#     anchors = init_points[:, indices]
    
#     new_anchors = anchors.copy() np.repeat(init_points[:, 0].reshape((2, 1))
    
#     if np.arccos(() / (np.linalg.norm(anchors)))
    
#     return anchors

In [24]:
num_frames = 1000

kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))

def filtrate_red(hsv_masks, clear_noise=True, show_filtered=False, min_perimeter=15):
    # Create a VideoCapture object and read from input file
    # If the input is the camera, pass 0 instead of the video file name
    
    cam = cv2.VideoCapture(0)

    # Check if camera opened successfully
    if (cam.isOpened()== False):
        print("Error opening video stream or file")

    # Get image size
    width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Read until video is completed
    for i in range(0, num_frames):
        if not cam.isOpened():
            break

        # Capture frame-by-frame
        ret, frame = cam.read()    
        if ret == True:
            hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            frame_threshed = np.zeros((height, width), dtype=np.uint8)
            for each_mask in hsv_masks:
                frame_threshed += cv2.inRange(hsv_img, each_mask[0], each_mask[1])

            if clear_noise:
                frame_threshed = cv2.morphologyEx(frame_threshed, cv2.MORPH_OPEN, kernel_open)
                frame_threshed = cv2.morphologyEx(frame_threshed, cv2.MORPH_CLOSE, kernel_close)
                
            _, contours, hierarchy = cv2.findContours(frame_threshed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            
            backtorgb = cv2.cvtColor(frame_threshed, cv2.COLOR_GRAY2RGB)
            
            if show_filtered:
                image_to_show = backtorgb
            else:
                image_to_show = frame
            
            for i in range(len(contours)):
                if len(contours[i]) > min_perimeter:
                    cv2.drawContours(image_to_show, contours, i, (0, 0, 255), 2)  
                    
                    ellipse = cv2.fitEllipse(contours[i])
                    cv2.ellipse(image_to_show, ellipse, (0, 255, 0), 2)
#                     print(ellipse)
            
#             if np.all(anchor_set):
#                 return image_to_show, anchor_points, default_anchors
#                 image_to_show = get_transformed_image(image_to_show, anchor_points, default_anchors)
                
            
            #  cv2.drawContours(backtorgb, contours, -1, (0, 0, 255), 2)  

            cv2.imshow('Frame', image_to_show)

            #  plt.imshow(backtorgb)
            
            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

        # Break the loop
        else: 
            break

    # When everything done, release the video capture object
    cam.release()

    # Closes all the frames
    cv2.destroyAllWindows()

In [22]:
# lower red mask (0-10)
# upper red mask (170-180)
hsv_masks = [[np.array([0, 110, 0], np.uint8), 
              np.array([10, 255, 255], np.uint8)],
             [np.array([160, 110, 0], np.uint8), 
              np.array([180, 255, 255], np.uint8)]]

In [23]:
filtrate_red(hsv_masks, clear_noise=True, show_filtered=False, min_perimeter=15)

(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)


# Main code

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import time
import serial

In [2]:
def get_hsv_masks(color='red'):
    if color == 'red':
        return [[np.array([0, 80, 0], np.uint8),
                 np.array([10, 255, 255], np.uint8)],
                [np.array([160, 80, 0], np.uint8), 
                 np.array([180, 255, 255], np.uint8)]]

    if color == 'green':
        return [[np.array([45, 100, 100], np.uint8), 
                 np.array([75, 255, 255], np.uint8)]]
        
    print('Color must be green or red')
    return None
    

In [8]:
def clear_binary_from_noise(image_threshed):
    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))

    image_threshed = cv2.morphologyEx(image_threshed, cv2.MORPH_OPEN, kernel_open)
    image_threshed = cv2.morphologyEx(image_threshed, cv2.MORPH_CLOSE, kernel_close)

In [9]:
def detect_colored_objects(hsv_img, min_perimeter, clear_noise=True, color='red'):
    height, width = hsv_img.shape[0:2]
    
    image_threshed = np.zeros((height, width), dtype=np.uint8)
    
    hsv_masks = get_hsv_masks(color)
    
    for each_mask in hsv_masks:
        image_threshed += cv2.inRange(hsv_img, each_mask[0], each_mask[1])

    if clear_noise:
        clear_binary_from_noise(image_threshed)

    _, contours, hierarchy = cv2.findContours(image_threshed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    circles = []
    for contour in contours:
        if len(contour) > min_perimeter:
            radius = np.sqrt(np.sum((contour[len(contour) // 2] - contour[0]) ** 2)) / 2
            circles.append([0.5 * (contour[0] + contour[len(contour) // 2]).reshape(2), radius])

    return circles

In [4]:
def process_frame(cam, min_perimeter, show_picture=True, clear_noise=True):
    # Initial processing

    ret, image = cam.read()    
    
    if not ret:
        print("Cannot read a frame")
        return 1
    
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    rodeo_circles = detect_colored_objects(hsv_img, min_perimeter, 
                                           clear_noise, color='red')
    obstacle_circles = detect_colored_objects(hsv_img, min_perimeter, 
                                              clear_noise, color='green')

    for circle in rodeo_circles:
        cv2.circle(image, tuple(circle[0].astype('int')), 
                   circle[1].astype('int'), (0, 0, 255), 2)
        
    for circle in obstacle_circles:
        cv2.circle(image, tuple(circle[0].astype('int')), 
                   circle[1].astype('int'), (0, 255, 0), 2)
    
    cv2.imshow('Frame', image)
    
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        return 1
    
    return 0

In [5]:
def send_decison():
    return

def make_decision():
    return

In [6]:
def run_rodeo(max_time=1000, min_perimeter=15):
    # Connect to the transmitter
    
    # Connect to a webcam
    
    cam = cv2.VideoCapture(0)
    if (cam.isOpened() == False):
        print("Error opening video stream or file")

#     width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
#     height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Process video
    for time in range(max_time):
        if not cam.isOpened():
            print("Camera is not opened")
            break
            
        ret = process_frame(cam, min_perimeter=min_perimeter)
        
        if ret:
            break
        
#         make_decision()
        
#         send_decision()
        
    cam.release()
    cv2.destroyAllWindows()

In [7]:
run_rodeo(max_time=10000, min_perimeter=10)

# Write data to Arduino

In [ ]:
from time import sleep
import serial

In [ ]:
ser = serial.Serial('COM9', 9600) # Establish the connection on a specific port

In [ ]:
ser.write(b'B') # Convert the decimal number to ASCII then send it to the Arduino

In [ ]:
print(ser.readline()) # Read the newest output from the Arduino
# sleep(0.1) # Delay for one tenth of a second

# Robot control

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import time
import serial
from time import sleep
import serial
from pynput import keyboard

## Keyboard control with pynput

In [ ]:
ser = serial.Serial('COM9', 9600)

def on_press(key):
    try:
        if 'w' == '{0}'.format(key)[1]:
            ser.write(b'w')
            sleep(0.0001)
                    
        if 'a' == '{0}'.format(key)[1]:
            ser.write(b'a')
            sleep(0.0001)

        if 's' == '{0}'.format(key)[1]:
            ser.write(b's')
            sleep(0.0001)

        if 'd' == '{0}'.format(key)[1]:
            ser.write(b'd')
            sleep(0.0001)
            
        if 'c' == '{0}'.format(key)[1]:
            ser.write(b'c')
            sleep(0.0001)
    except AttributeError:
        pwrint('special key {0} pressed'.format(key))

def on_release(key):
#     print('{0} released'.format(key))
    if key == keyboard.Key.esc:
        # Stop listener
        return False

# Collect events until released
with keyboard.Listener(
        on_press=on_press,
        on_release=on_release) as listener:
    listener.join()
    
ser.close()

## Keyboard control with OpenCV

In [ ]:
cv2.imshow('Frame', np.zeros((200, 200)))
while (True):
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
                    
    if cv2.waitKey(25) & 0xFF == ord('w'):
        ser.write(b'w')
        sleep(0.001)
                    
    if cv2.waitKey(25) & 0xFF == ord('a'):
        ser.write(b'a')
        sleep(0.001)
                    
    if cv2.waitKey(25) & 0xFF == ord('s'):
        ser.write(b's')
        sleep(0.001)
                    
    if cv2.waitKey(25) & 0xFF == ord('d'):
        ser.write(b'd')
        sleep(0.001)

ser.close()
cv2.destroyAllWindows()